#Engagement Rate

In [26]:
import pandas as pd

# Load DataFrames from CSV files
df_members = pd.read_csv('D:\Data_Analyst\Project_Assignment\Technical_Round_1_Analytics_Proposal\data\member_info.csv')  # Member Info
df_messages = pd.read_csv('D:\Data_Analyst\Project_Assignment\Technical_Round_1_Analytics_Proposal\data\message_info.csv')  # Message Info
df_groups = pd.read_csv('D:\Data_Analyst\Project_Assignment\Technical_Round_1_Analytics_Proposal\data\group_info.csv')  # Group Info

engagement_results = []

# Iterate through each group_id in the groups DataFrame
for group_id in df_groups['group_id'].unique():
    # 1. Filter Messages for the Current Group
    group_messages = df_messages[df_messages['group_id'] == group_id]
    
    # 2. Identify Active Members (Those who have sent at least one message)
    active_members = group_messages['sender_id'].unique()
    num_active_members = len(active_members)
    
    # 3. Total Members in the Group
    total_members = df_members[df_members['group_id'] == group_id].shape[0]
    
    # 5. Get Group Name
    group_name = df_groups[df_groups['group_id'] == group_id]['title'].values[0]
        
    # 4. Calculate Engagement Rate
    if total_members > 0:  # Avoid division by zero
        engagement_rate = round((num_active_members / total_members) * 100)
    else:
        engagement_rate = None  # Indicates no members in the group
    
    # 5. Append Results to the List
    engagement_results.append({
        'group_id': group_id,
        'group_name': group_name,
        'num_active_members': num_active_members,
        'total_members': total_members,
        'engagement_rate': engagement_rate
    })

# Convert Results to a DataFrame
engagement_results_df = pd.DataFrame(engagement_results)

# Display the Results
print("Highest engagement result in particular group:\n", engagement_results_df.sort_values(by=["engagement_rate"], ascending=False))
print("\nLowest engagement result in particular group:\n", engagement_results_df.sort_values(by=["engagement_rate"], ascending=True))

Highest engagement result in particular group:
      group_id      group_name  num_active_members  total_members  \
1           2   message Group                   7              1   
64         65      keep Group                   7              1   
123       124      full Group                   7              1   
183       184     order Group                   6              1   
171       172   general Group                   5              1   
..        ...             ...                 ...            ...   
23         24    single Group                   2             10   
140       141   current Group                   1              6   
162       163    simply Group                   1              6   
138       139  everyone Group                   0              4   
121       122   suggest Group                   8              0   

     engagement_rate  
1              700.0  
64             700.0  
123            700.0  
183            600.0  
171            500.0

In [28]:
results = []

# Iterate through each group_id in the groups DataFrame
for group_id in df_groups['group_id'].unique():
    # 1. Filter Members Belonging to the Current Group
    group_members = df_members[df_members['group_id'] == group_id]
    
    # 2. Count Bots and Humans
    num_bots = group_members[group_members['is_bot'] == True].shape[0]
    num_humans = group_members[group_members['is_bot'] == False].shape[0]
    
    # 3. Calculate Bot-to-Human Ratio
    if num_humans > 0:  # Avoid division by zero
        bot_to_human_ratio = round((num_bots / num_humans) * 100)
    else:
        bot_to_human_ratio = None  # Indicates no human members in the group
    
    # 4. Append Results to the List
    results.append({
        'group_id': group_id,
        'num_bots': num_bots,
        'num_humans': num_humans,
        'bot_to_human_ratio': bot_to_human_ratio
    })

# Convert Results to a DataFrame
results_df = pd.DataFrame(results)

# Display the Results
print(results_df.sort_values(by=["bot_to_human_ratio"], ascending=False))

     group_id  num_bots  num_humans  bot_to_human_ratio
11         12         4           1               400.0
19         20         7           2               350.0
10         11         3           1               300.0
16         17         3           1               300.0
22         23         6           2               300.0
..        ...       ...         ...                 ...
121       122         0           0                 NaN
123       124         1           0                 NaN
138       139         4           0                 NaN
143       144         2           0                 NaN
171       172         1           0                 NaN

[200 rows x 4 columns]


In [32]:
# Ensure that 'timestamp' is properly converted to datetime format
# We'll first check if 'timestamp' is already in datetime format, if not, convert it.
df_messages['timestamp'] = pd.to_datetime(df_messages['timestamp'], errors='coerce')

# Handle any NaT (Not a Time) values that may have resulted from invalid date formats
if df_messages['timestamp'].isna().sum() > 0:
    print(f"Warning: {df_messages['timestamp'].isna().sum()} invalid timestamps found and converted to NaT.")

# Initialize a list to store results
churn_results = []

# Iterate through each group_id in the groups DataFrame
for group_id in df_groups['group_id'].unique():
    # 1. Filter Members Belonging to the Specific Group
    group_members = df_members[df_members['group_id'] == group_id]
    
    # 2. Identify Members Who Left: Assume that members who haven't posted in a certain time period are 'inactive' or 'left'
    last_active_date = df_messages[df_messages['group_id'] == group_id]['timestamp'].max()  # Last message timestamp in the group
    
    if pd.isna(last_active_date):  # If there are no messages for this group, handle gracefully
        print(f"Warning: No messages found for Group {group_id}. Skipping churn rate calculation.")
        continue
    
    threshold_date = pd.to_datetime(last_active_date) - pd.DateOffset(days=30)  # Churn period: last 30 days
    
    # Find members who did not send messages after the threshold date
    inactive_members = group_members[~group_members['user_id'].isin(
        df_messages[df_messages['timestamp'] > threshold_date]['sender_id'].unique()
    )]
    num_left = len(inactive_members)
    
    # 3. Calculate Churn Rate
    total_members = group_members.shape[0]
    if total_members > 0:
        churn_rate = (num_left / total_members) * 100
    else:
        churn_rate = None  # If no members exist, return None
    
    # 4. Get Group Name
    group_name = df_groups[df_groups['group_id'] == group_id]['title'].values[0]
    
    # 5. Append Results to the List
    churn_results.append({
        'group_id': group_id,
        'group_name': group_name,
        'num_left': num_left,
        'total_members': total_members,
        'churn_rate': churn_rate
    })

# Convert Results to a DataFrame
churn_results_df = pd.DataFrame(churn_results)

# Display the Results
print(churn_results_df)

# Optional: Save Results to a CSV File
churn_results_df.to_csv('member_churn_analysis_with_group_name.csv', index=False)


     group_id      group_name  num_left  total_members  churn_rate
0           1     shake Group         6              9   66.666667
1           2   message Group         1              1  100.000000
2           3      firm Group         2              8   25.000000
3           4  military Group         1              5   20.000000
4           5     sense Group         1              3   33.333333
..        ...             ...       ...            ...         ...
194       196    itself Group         1              4   25.000000
195       197       her Group         2              4   50.000000
196       198    others Group         5              8   62.500000
197       199  pressure Group         3              7   42.857143
198       200      very Group         1              6   16.666667

[199 rows x 5 columns]
